In [1]:
import requests
import pandas as pd
# from pandas.io.json import json_normalize
from config import *
import sqlite3
from sqlite3 import Error

### Yelp

In [2]:
yelp_restaurant = requests.get('https://api.yelp.com/v3/businesses/search',
            params = {
                'location': 'Vancouver, BC',
                'term': 'restaurants'},
            headers = {'Authorization': f'Bearer {YELP_API}'})

In [3]:
yelp_restaurant.status_code

200

### Yelp

In [51]:
df_yelp = pd.json_normalize(yelp_restaurant.json(), 'businesses')
df_yelp.to_csv('data/yelp_restaurants.csv')

In [62]:
pop_yelp = df_yelp[['name', 'rating', 'review_count']]

### Top 10 Ratings

In [63]:
top10_rating_yelp = pop_yelp.head(10)
top10_rating_yelp.sort_values('rating', ascending=False)

,name,rating,review_count
0,Miku,4.5,1883
1,Fable Kitchen,4.5,693
2,Kingyo,4.5,888
4,Guu with Garlic,4.5,946
6,Saku Robson,4.5,214
8,Blue Water Cafe,4.5,939
9,Kosoo,4.5,27
3,The Flying Pig - Yaletown,4.0,1120
5,Dinesty Dumpling House,4.0,805
7,Saku Broadway,4.0,150


### Bottom 10 Ratings

In [64]:
bot10_rating_yelp = pop_yelp.tail(10)
bot10_rating_yelp.sort_values('rating', ascending=False)

,name,rating,review_count
13,Fanny Bay Oyster Bar & Shellfish Market,4.5,603
15,Linh Cafe,4.5,17
17,Jam Cafe on Beatty,4.5,1146
19,So Hyang Korean Cuisine,4.5,284
10,Phnom Penh,4.0,1356
11,Chambar,4.0,1396
12,Medina Cafe,4.0,2364
14,Twisted Fork,4.0,1052
16,The Flying Pig - Gastown,4.0,943
18,Forage,4.0,645


### Summary

In [65]:
pop_yelp.describe()

,rating,review_count
count,20.000000,20.000000
mean,4.275000,873.550000
std,0.255209,600.292602
min,4.000000,17.000000
25%,4.000000,523.250000
50%,4.500000,913.500000
75%,4.500000,1126.500000
max,4.500000,2364.000000


### Top 10 Reviews

In [66]:
top10_review_yelp = pop_yelp.head(10)
top10_review_yelp.sort_values('review_count', ascending = False)

,name,rating,review_count
0,Miku,4.5,1883
3,The Flying Pig - Yaletown,4.0,1120
4,Guu with Garlic,4.5,946
8,Blue Water Cafe,4.5,939
2,Kingyo,4.5,888
5,Dinesty Dumpling House,4.0,805
1,Fable Kitchen,4.5,693
6,Saku Robson,4.5,214
7,Saku Broadway,4.0,150
9,Kosoo,4.5,27


### Bottom 10 Reviews

In [67]:
bot10_review_yelp = pop_yelp.tail(10)
bot10_review_yelp.sort_values('review_count')

,name,rating,review_count
15,Linh Cafe,4.5,17
19,So Hyang Korean Cuisine,4.5,284
13,Fanny Bay Oyster Bar & Shellfish Market,4.5,603
18,Forage,4.0,645
16,The Flying Pig - Gastown,4.0,943
14,Twisted Fork,4.0,1052
17,Jam Cafe on Beatty,4.5,1146
10,Phnom Penh,4.0,1356
11,Chambar,4.0,1396
12,Medina Cafe,4.0,2364


### FourSquare

In [97]:
fsq_restaurant = requests.get('https://api.foursquare.com/v3/places/search',
                              params = {
                                  'near': 'Vancouver, BC',
                                  'categories': '13065',
                                  'limit': 50
                              },
                              headers = {
                                  'Accept': 'application/json',
                                  'Authorization': 'fsq3NBY1z05TqyRLxaFXhEmuBXlW4C+oCQ8jFzDNErgRGfs='
                              })

In [99]:
fsq_restaurant.status_code

200

In [100]:
df_fsq = pd.json_normalize(fsq_restaurant.json(), 'results')
df_fsq.to_csv('data/fsq_restaurant.csv')

In [250]:
fsq_data = pd.read_csv('foursquare_data.csv')
df_fsq = pd.DataFrame(fsq_data)
df_fsq

,Name,Latitude,Longitude,Likes,City,State,Category
0,Warehouse31,33.296218,-86.813072,9,Pelham,AL,Arts/Entertainment
1,Cowboy Cheerleading,33.270635,-86.797635,0,Pelham,AL,Arts/Entertainment
2,Pelham Civic Complex,33.329325,-86.788621,10,Pelham,AL,Arts/Entertainment
3,Jelly's Bar,33.309999,-86.802501,3,Pelham,AL,Arts/Entertainment
4,Task 3,33.251538,-86.828777,0,Pelham,AL,Arts/Entertainment
...,...,...,...,...,...,...,...
101,King's Crest Lookout,33.275789,-86.759066,0,Pelham,AL,Outdoor/Rec
102,YMCA Pool,33.319859,-86.798235,1,Pelham,AL,Outdoor/Rec
103,gym time,33.241736,-86.817824,1,Pelham,AL,Outdoor/Rec
104,Ballantrae Club House,33.271496,-86.754196,6,Pelham,AL,Outdoor/Rec


In [180]:
df_fsq.groupby('Category').size()

Category
Arts/Entertainment    25
Food                  30
Nightlife             24
Outdoor/Rec           27
dtype: int64

In [273]:
pop_fsq = df_fsq[df_fsq['Category'] == 'Food'][['Name', 'Likes']]
pop_fsq.groupby('Name').sum().sort_values('Likes', ascending = False).head(10)

,Likes
Name,
Publix,82
Joe's Italian,68
Cozumel Grill & Mexican Restaurant,52
The Depot Deli & Grill,33
Donut Joe's,30
Jack's,29
Sonic Drive-In,19
Zaxby's Chicken Fingers & Buffalo Wings,19
Hibachi Super Buffet,18


In [292]:
pop_yelp.sort_values(['rating', 'review_count'], ascending = False).head(10)

,name,review_count,rating
0,Smiley Brothers Specialty Foods,67,5.0
17,Blue Pacific At Hoover Foodmart,284,4.5
16,Baha Burger,280,4.5
6,Taqueria Garibaldi,75,4.5
1,Donut Joe's,72,4.5
2,Oak Mountain State Park,72,4.5
3,The Beer Hog,69,4.5
13,Ooka Chinese & Japanese Restaurant,69,4.5
8,Pelham Diner,59,4.5
9,La Libertad,40,4.5


## SQL

In [103]:
def create_connection(path):
    try:
        connection = sqlite3.connect(path)
        print('Connection to SQLite DB successful')
    except Error as e:
        print(f'The error "{e}" occurred')
    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print('Query executed successfully')
    except Error as e:
        print(f'The error {e} occurred')

In [104]:
connection = create_connection("lhl_project_2.sqlite")

Connection to SQLite DB successful


In [105]:
create_restaurant_table = '''
CREATE TABLE IF NOT EXISTS restaurant (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    review_count INTEGER,
    rating FLOAT,
    address_1 TEXT,
    address_2 TEXT,
    city TEXT,
    postal_code TEXT)
'''

In [106]:
execute_query(connection, create_restaurant_table)

Query executed successfully


## Run out of query

In [19]:
location = 'Vancouver'
v = '20200731'

In [106]:
fsq_url = requests.get('https://api.foursquare.com/v2/venues/search?near=' + location + '&client_id=' + fsq_client_id + '&client_secret=' + fsq_client_secret + '&v=' + v)

In [147]:
fsq_detail = requests.get('https://api.foursquare.com/v2/venues/4ae7afb2f964a52058ad21e3?client_id=' + fsq_client_id + '&client_secret=' + fsq_client_secret + '&v=' + v)

In [107]:
fsq_url

<Response [200]>

In [127]:
fsq_detail

<Response [200]>

In [113]:
fsq_json = fsq_url.json()
df_fsq = pd.json_normalize(fsq_json, record_path = ['response', 'venues'])
df_fsq.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode
0,4ab56c01f964a520857420e3,King Edward SkyTrain Station,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",v-1622825860,False,500 W King Edward Ave.,at Cambie St.,49.249120,-123.115703,"[{'label': 'display', 'lat': 49.24912042946037...",CA,Vancouver,BC,Canada,"[500 W King Edward Ave. (at Cambie St.), Vanco...",NaN
1,5a481d41610f044748e5e260,The Hobbit House,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",v-1622825860,False,587 W King Edward Ave,NaN,49.249083,-123.116835,"[{'label': 'display', 'lat': 49.249083, 'lng':...",CA,Vancouver,BC,Canada,"[587 W King Edward Ave, Vancouver BC V5Z, Canada]",V5Z
2,5cf4eb74cabcff002c2ef9b3,Blue Hat Bistro,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1622825860,False,1505 #101 West 2nd Ave.,(Entrance on Anderson Street),49.269970,-123.137713,"[{'label': 'display', 'lat': 49.26996950868134...",CA,Vancouver,BC,Canada,[1505 #101 West 2nd Ave. ((Entrance on Anderso...,V6H 3Y4
3,4ae7afb2f964a52058ad21e3,Landmark Hot Pot House 春秋火鍋,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1622825860,False,4023 Cambie St.,at W. King Edward Ave.,49.249759,-123.115515,"[{'label': 'display', 'lat': 49.24975945918286...",CA,Vancouver,BC,Canada,"[4023 Cambie St. (at W. King Edward Ave.), Van...",V5Z 2X9‎
4,5be40c66b25fee002c19e60f,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1622825860,False,"990 25 Ave W, Safeway Grocery Store",NaN,49.248601,-123.126520,"[{'label': 'display', 'lat': 49.248601, 'lng':...",CA,Vancouver,BC,Canada,"[990 25 Ave W, Safeway Grocery Store, Vancouve...",V5Z 2E2


In [146]:
pop_fsq = df_fsq[['name', 'id']]
print(len(df_fsq))
pop_fsq = pop_fsq.head(100)

30


In [144]:
ratings = []


for i in range(0, len(pop_fsq['id'])):
#     print(f'This is row {i}')
    
    fsq_id = pop_fsq['id'][i]
    try:
        fsq_rating = requests.get('https://api.foursquare.com/v2/venues/' + fsq_id + '?client_id=' + fsq_client_id + '&client_secret=' + fsq_client_secret + '&v=' + v)

        fsq_rating_json = fsq_rating.json()
        rating = fsq_rating_json['response']['venue']['rating']
    
        ratings.append(rating)
        
    except KeyError: 
#         print('No rating for this ID')
        ratings.append('NA')

In [145]:
print(ratings)

['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']


In [ ]:
pop_fsq = pd.Series(ratings)

# df_fsq['rating'] = pd.Series(ratings)

In [149]:
fsq_detail_json = fsq_detail.json()
print(fsq_detail_json)
# fsq_detail_json['response']['venue']['rating']

# df_fsq_detail = pd.json_normalize(fsq_detail_json, record_path = ['response', 'venue'])
# df_fsq.head(10)

# pop_fsq = df_fsq[['name', 'id']]
# pop_fsq

{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '60ba7b7fea5de365004a1873'}, 'response': {}}
